In [ ]:
# Setting GPU Environment
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

print("start")

In [1]:
# Author: Chandranil Chakraborttii
# Project: Learning I/O Access Patterns to Improve Prefetching in SSDs
# Paper Link :https://www.researchgate.net/profile/Chandranil_Chakraborttii/publication/344379801_Learning_IO_Access_Patterns_to_Improve_Prefetching_in_SSDs/links/5f6e28fba6fdcc00863adb13/Learning-I-O-Access-Patterns-to-Improve-Prefetching-in-SSDs.pdf
# Loading libraries

from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
# import pylru



start


In [2]:
import collections

# Hyper parameters to set     K = 15 and B = 128
# Baselines: Stride and RepeatMax

    
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros
    

def stride(f):
    df = pd.read_csv(f,engine='python',skiprows =3,header=None,na_values=['-1'], index_col=False)
    df.columns = names
    print(len(df))
    df = df.sort_values(by=['TimeStamp'])
    df.reset_index(inplace=True, drop=True)
    df = df[df.ByteOffset != 'ByteOffset'] # Removing extra rows
    addresses = df['ByteOffset'].tolist()

    #  Buffer Size = 1
    K = 15
    B = 128
    count = 1
    hit = 0
    hit_1 = 0
    hit_2 = 0
    prefetches = 0
    bit_length = '064b'

    # Create a deque of N- elements
    buffer_size = 10
    init_list = zerolistmaker(buffer_size)
    buffer = collections.deque(init_list)
    
    buffer_size_1 = 100
    init_list_1 = zerolistmaker(buffer_size_1)
    buffer_1 = collections.deque(init_list_1)

    buffer_size_2 = 1000
    init_list_2 = zerolistmaker(buffer_size_2)
    buffer_2 = collections.deque(init_list_2)

    bucket = []
    history = [0,0,0]
    addresses = df['ByteOffset'].tolist()
    for i in range(B):
        bucket.append(history)

    while (count < len(addresses)-1):
        lba_dec = addresses[count]
        
        if lba_dec in buffer:
            hit += 1
        if lba_dec in buffer_1:
            hit_1 += 1
        if lba_dec in buffer_2:
            hit_2 += 1
            
        lba_binary = str(format(lba_dec, bit_length))
        S = int(lba_binary[:K])
        S_ = lba_binary[K-1:]
        pos = S % B

        #Organizing in a list
        bucket[pos][2] = bucket[pos][1]
        bucket[pos][1] = bucket[pos][0]
        bucket[pos][0] = str(lba_dec)

        delta1 = int(bucket[pos][2]) - int(bucket[pos][1])
        delta2 = int(bucket[pos][1]) - int(bucket[pos][0])
        if(delta1 == delta2):
            buffer.append(lba_dec + delta1)
            buffer.popleft()
            buffer_1.append(lba_dec + delta1)
            buffer_1.popleft()
            buffer_2.append(lba_dec + delta1)
            buffer_2.popleft()
            prefetches += 1

       
        count = count+1 

    print("Prefetch Coverage: ",  str(prefetches/count))
    print(" For Buffer Size = ", str(buffer_size))
    print("The prefetch hit ratio is =", str(hit/count))
    print("****************************************************************")

    print("Buffer Size = ", str(buffer_size_1))
    print("The prefetch hit ratio is =", str(hit_1/count))

    print("****************************************************************")
    
    print("Buffer Size = ", str(buffer_size_2))
    print("The prefetch hit ratio is =", str(hit_2/count))
    print("****************************************************************")


In [18]:
for f in all_files:
    if "2016031115-LUN2" in f:
        print(f)
        stride(f)
    
    

/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/part32/2016031115-LUN2.csv
2408226
Prefetch Coverage:  0.022999927332371353
 For Buffer Size =  10
The prefetch hit ratio is = 0.00060044223442577
****************************************************************
Buffer Size =  100
The prefetch hit ratio is = 0.0013918965212968057
****************************************************************
Buffer Size =  1000
The prefetch hit ratio is = 0.005188053441850325
****************************************************************


In [17]:
f = r'/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/SYSTOR/part26/2016030817-LUN2.csv'
stride(f)

4443485
Prefetch Coverage:  0.01826832278455374
 For Buffer Size =  10
The prefetch hit ratio is = 0.00010037169032227865
****************************************************************
Buffer Size =  100
The prefetch hit ratio is = 0.00028356127759208765
****************************************************************
Buffer Size =  1000
The prefetch hit ratio is = 0.0014794697134050668
****************************************************************
